In [1]:
import os
import json
import pandas as pd

from autumn.settings.folders import PROJECTS_PATH
from autumn.core.inputs.database import get_input_db
from autumn.settings.constants import COVID_BASE_DATETIME


In [2]:
input_db = get_input_db()

iso3s = ["PHL"]
timeseries = {}
for iso3 in iso3s:
    data = input_db.query(
        table_name='owid', 
        conditions= {"iso_code": iso3},
        columns=["date", "new_deaths"]
    )

    data["smoothed_new_deaths"] = data["new_deaths"].rolling(14).mean()
    data.dropna(inplace=True)

    timeseries["infection_deaths"] = {
        "output_key": "infection_deaths",
        "times": (pd.to_datetime(data["date"])- COVID_BASE_DATETIME).dt.days.to_list(),
        "values": data["smoothed_new_deaths"].to_list(),
        "quantiles": [.025, .25, .5, .75, .975]
    }

    targets_path = os.path.join(PROJECTS_PATH, "sm_covid", "philippines", "timeseries.json")
 
    # Writing to sample.json
    with open(targets_path, "w") as outfile:
        json.dump(timeseries, outfile, indent=2)
